## Chapter 3: Tranportation problem

In [1]:
from amplpy import AMPL
ampl = AMPL()
ampl.option['solver'] = 'highs'

Model specification

In [9]:
ampl.reset()

# set of origins representing the production mills
ampl.eval('''set ORIG;''')
# set of destinations representing factories 
ampl.eval('''set DEST;''')
# set of supply values for each origin
ampl.eval('''param supply {ORIG} >= 0;''')
# set of demand values for each destination
ampl.eval('''param demand {DEST} >= 0;''')
# the sum of the supplies has to equal the sum of the demands
ampl.eval('''check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];''')
# cost of shipping from each origin to each destination
ampl.eval('''param cost {ORIG, DEST} >= 0;''')
# amount transported from each origin to each destination
ampl.eval('''var trans {ORIG, DEST} >= 0;''')
# objective function: minimize the total cost of transportation
ampl.eval('''minimize Total_Cost:
          sum {i in ORIG, j in DEST} cost[i,j] * trans[i,j];''')
# the sum of the transported goods from each origin has to be equal to the supply
ampl.eval('''subject to Supply {i in ORIG}: sum{j in DEST} trans[i,j] = supply[i];''')
# the sum of the transported goods to each destination has to be equal to the demand
ampl.eval('''subject to Demand {j in DEST}: sum {i in ORIG} trans[i,j] = demand[j];''')


	line 1 offset 60
	i is not defined
	context:  subject to Demand {j in DEST}: sum {i in ORIG} trans[i,j] =  >>> demand[i] <<< ;


Data specification

In [14]:
origin = ['GARY', 'CLEV', 'PITT']
supply = [1400, 2600, 2900]
destination = ['FRA', 'DET', 'LAN', 'WIN', 'STL', 'FRE', 'LAF']
demand = [900, 1200, 600, 400, 1700, 1100, 1000]
cost = [[39, 14, 11, 14, 16, 82,  8],
        [27,  9, 12,  9, 26, 95, 17],
        [24, 14, 17, 13, 28, 99, 20]]
ampl.set['ORIG'] = origin
ampl.param['supply'] = supply
ampl.set['DEST'] = destination
ampl.param['demand'] = demand
for orig in origin:
    for dest in destination:
        ampl.param['cost'][orig, dest] = cost[origin.index(orig)][destination.index(dest)]

Display the problem instance

In [15]:
ampl.eval('expand;')

minimize Total_Cost:
	39*trans['GARY','FRA'] + 14*trans['GARY','DET'] + 
	11*trans['GARY','LAN'] + 14*trans['GARY','WIN'] + 
	16*trans['GARY','STL'] + 82*trans['GARY','FRE'] + 8*trans['GARY','LAF']
	 + 27*trans['CLEV','FRA'] + 9*trans['CLEV','DET'] + 
	12*trans['CLEV','LAN'] + 9*trans['CLEV','WIN'] + 26*trans['CLEV','STL']
	 + 95*trans['CLEV','FRE'] + 17*trans['CLEV','LAF'] + 
	24*trans['PITT','FRA'] + 14*trans['PITT','DET'] + 
	17*trans['PITT','LAN'] + 13*trans['PITT','WIN'] + 
	28*trans['PITT','STL'] + 99*trans['PITT','FRE'] + 
	20*trans['PITT','LAF'];

subject to Supply['GARY']:
	trans['GARY','FRA'] + trans['GARY','DET'] + trans['GARY','LAN'] + 
	trans['GARY','WIN'] + trans['GARY','STL'] + trans['GARY','FRE'] + 
	trans['GARY','LAF'] = 1400;

subject to Supply['CLEV']:
	trans['CLEV','FRA'] + trans['CLEV','DET'] + trans['CLEV','LAN'] + 
	trans['CLEV','WIN'] + trans['CLEV','STL'] + trans['CLEV','FRE'] + 
	trans['CLEV','LAF'] = 2600;

subject to Supply['PITT']:
	trans['PITT','FRA'] + tr

Solve the problem instance

In [16]:
ampl.solve()

HiGHS 1.7.1:HiGHS 1.7.1: optimal solution; objective 72300
0 simplex iterations
0 barrier iterations


Display solution

In [17]:
ampl.display('trans')

trans [*,*] (tr)
:     CLEV   GARY   PITT    :=
DET   2600      0      0
FRA      0      0      0
FRE      0      0      0
LAF      0   1400      0
LAN      0      0      0
STL      0      0      0
WIN      0      0   2900
;

